In [3]:
import pathlib
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.ndimage
import scipy.signal
import skimage
import skimage.io as io
import skimage.morphology
import skimage.segmentation
from centrosome.cpmorphology import fixup_scipy_ndimage_result as fix
from scipy.linalg import lstsq

sys.path.append("../featurization")
import numpy
from area_size_shape import calulate_surface_area, measure_3D_area_size_shape
from colocalization import calculate_3D_colocalization
from data_writer import organize_featurization_data
from granularity import measure_3D_granularity
from intensity import measure_3D_intensity
from loading_classes import Featurization, ImageSetLoader, ObjectLoader
from neighbors import measure_3D_number_of_neighbors
from texture import measure_3D_texture
from two_object_loading_classes import TwoObjectImageSetLoader

In [4]:
channel_mapping = {
    "nuclei": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "nuclei_mask": "nuclei_",
    "cell_mask": "cell_",
    "cytoplasm_mask": "cytoplasm_",
    "organoid_mask": "organoid_",
}

In [5]:
image_set_path = pathlib.Path("../../data/NF0014/cellprofiler/C4-2/")

In [9]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)
image_set_loader.retrieve_image_attributes()
print(image_set_loader.image_names)
print(image_set_loader.unique_compartment_objects.keys())
print(image_set_loader.compartments)

['nuclei', 'AGP', 'ER', 'Mito', 'BF']
dict_keys(['cell_mask', 'cytoplasm_mask', 'nuclei_mask', 'organoid_mask'])
['cell_mask', 'cytoplasm_mask', 'nuclei_mask', 'organoid_mask']


In [45]:
# get all combinations of channels, compartments, and objects
unique_object = 1
object_compartment_channel_measuremnt_pair = []
for channel in image_set_loader.image_names:
    for compartment in image_set_loader.compartments:
        object_compartment_channel_measuremnt_pair.append((channel, compartment))

# combination of object_compartment_channel_pair
combinations = []
import itertools

combinations = list(
    itertools.combinations(object_compartment_channel_measuremnt_pair, 2)
)

print(len(combinations))
combinations

channel1, compartment1 = combinations[0][0]
channel2, compartment2 = combinations[0][1]

190


In [46]:
coloc_class = ColocalizationTwoObject_Loader(
    image_set_loader=image_set_loader,
    image1=image_set_loader.image_set_dict[channel1],
    label_image1=image_set_loader.image_set_dict[compartment1],
    compartment1=compartment1,
    object1=unique_object,
    image2=image_set_loader.image_set_dict[channel2],
    label_image2=image_set_loader.image_set_dict[compartment2],
    compartment2=compartment2,
    object2=unique_object,
)

In [47]:
featurizer = ColocalizationFeaturization(
    image_set_loader=image_set_loader,
    two_object_loader=coloc_class,
)
features = featurizer.process_features_for_output()
features

/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/2.cellprofiling/notebooks/../featurization/colocalization.py:161: RuntimeWarning: invalid value encountered in divide
  corr = scipy.ndimage.sum(x * y / (std1 * std2))
/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/2.cellprofiling/notebooks/../featurization/colocalization.py:180: RuntimeWarning: invalid value encountered in divide
  M1 = scipy.ndimage.sum(fi_thresh) / numpy.array(tot_fi_thr)
/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/2.cellprofiling/notebooks/../featurization/colocalization.py:181: RuntimeWarning: invalid value encountered in divide
  M2 = scipy.ndimage.sum(si_thresh) / numpy.array(tot_si_thr)
/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/2.cellprofiling/notebooks/../featurization/colocalization.py:195: RuntimeWarning: invalid value encountered in scalar divide
  scipy.ndimage.sum(
/home/lippincm/Documents/GFF_3D_organoid_profiling_pipeline/2.cellprofiling/notebooks/

{'colocalization_Mean Correlation coeff': nan,
 'colocalization_Median Correlation coeff': nan,
 'colocalization_Min Correlation coeff': nan,
 'colocalization_Max Correlation coeff': nan,
 'colocalization_Mean Manders coeff_M1': nan,
 'colocalization_Median Manders coeff_M1': nan,
 'colocalization_Min Manders coeff_M1': nan,
 'colocalization_Max Manders coeff_M1': nan,
 'colocalization_Mean Manders coeff_M2': nan,
 'colocalization_Median Manders coeff_M2': nan,
 'colocalization_Min Manders coeff_M2': nan,
 'colocalization_Max Manders coeff_M2': nan,
 'colocalization_Mean_overlap_coeff': nan,
 'colocalization_Median_overlap_coeff': nan,
 'colocalization_Min_overlap_coeff': nan,
 'colocalization_Max_overlap_coeff': nan,
 'colocalization_Mean_K1': nan,
 'colocalization_Median_K1': nan,
 'colocalization_Min_K1': nan,
 'colocalization_Max_K1': nan,
 'colocalization_Mean_K2': nan,
 'colocalization_Median_K2': nan,
 'colocalization_Min_K2': nan,
 'colocalization_Max_K2': nan,
 'colocalization